In [1]:
import pandas as pd
import glob

In [ ]:
# Carpeta - data diaria
ruta = "data/raw/osinergmin/"  
archivos = sorted(glob.glob(ruta + "osinergmin_*.csv"))

print(f"Archivos: {len(archivos)}")

dfs = []

for archivo in archivos:
    try:
        df = pd.read_csv(archivo)
        df["__origen__"] = archivo  # para rastrear el archivo
        dfs.append(df)
    except Exception as e:
        print(f"Error leyendo {archivo}: {e}")

# Concatenar todo
osinerg = pd.concat(dfs, ignore_index=True)

# Remover duplicados por si existen repeticiones
osinerg = osinerg.drop_duplicates()
# osinerg.to_csv("data/consolidated/combustibles/osinergmin.csv", index=False)
osinerg


Archivos: 19


,Distrito,Establecimiento,Dirección,Teléfono,Precio de Venta (Soles por galón),Departamento,Provincia,Producto,Fecha,Precio,__origen__
0,ANCON,CENTER GAS SAC,"PANAMERICANA NORTE KM 44.5, SUB LOTE A-1 ASOCI...",016788937/948581946/981530091,13.35,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.35,data/raw/osinergmin\osinergmin_01_11_2025.csv
1,ANCON,SERVICIOS DE COMBUSTIBLES SANTA FE E.I.R.L.,"AA.HH. LAS PALMERAS MZ. C, LT. 14 (ANTES SERPE...",016983211/960690748/996437408,13.35,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.35,data/raw/osinergmin\osinergmin_01_11_2025.csv
2,ANCON,ESTACIÓN DE SERVICIOS ANCÓN S.A.,"PANAMERICANA NORTE KM. 44, URB. SAN JOSE, MZ. ...",945635871/993487080,13.6,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.60,data/raw/osinergmin\osinergmin_01_11_2025.csv
3,ANCON,EMPRESA DE TRANSPORTES Y SERVICIOS VIRGEN DE L...,AV. LA FLORIDA CDRA. 12 MIRAMAR . LAS CONCHITAS,967743212,13.79,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.79,data/raw/osinergmin\osinergmin_01_11_2025.csv
4,ANCON,INVERSIONES RAMSAN EIRL,JR. PEDRO GAREZON N° 500,934124950,13.79,LIMA,LIMA,DB5 S-50 UV,2025-11-01,13.79,data/raw/osinergmin\osinergmin_01_11_2025.csv
...,...,...,...,...,...,...,...,...,...,...,...
37469,VILLA MARIA DEL TRIUNFO,SERVICENTRO MARCO ANTONIO S.R.LTDA.,AV. VILLA MARIA Nº 1400,NaN,15.29,LIMA,LIMA,Gasohol Regular,2025-10-31,15.29,data/raw/osinergmin\osinergmin_31_10_2025.csv
37470,VILLA MARIA DEL TRIUNFO,ESTACION DE SERVICIOS LUCOCIZA S.A.C.,AV. PACHACUTEC N° 3601,012812222,15.39,LIMA,LIMA,Gasohol Regular,2025-10-31,15.39,data/raw/osinergmin\osinergmin_31_10_2025.csv
37471,VILLA MARIA DEL TRIUNFO,E & A INVERSIONES S.A.,AV. PACHACUTEC N° 5955,012951742,15.89,LIMA,LIMA,Gasohol Regular,2025-10-31,15.89,data/raw/osinergmin\osinergmin_31_10_2025.csv
37472,VILLA MARIA DEL TRIUNFO,GRIFOS ESPINOZA S.A.,AV. PROLONGACION PACHACUTEC N° 5295,017080700/994295657,15.99,LIMA,LIMA,Gasohol Regular,2025-10-31,15.99,data/raw/osinergmin\osinergmin_31_10_2025.csv


In [ ]:
consolidado

In [ ]:
df = consolidado

In [ ]:
import pandas as pd

# Asegurar que Fecha sea datetime
df["Fecha"] = pd.to_datetime(df["Fecha"])

# Agrupar por día y calcular mediana y volatilidad (std)
daily = df.groupby("Fecha").agg(
    mediana_precio = ("Precio", "median"),
    volatilidad    = ("Precio", "std")
).reset_index()

# Bandas de confianza (1 std)
daily["banda_sup"] = daily["mediana_precio"] + daily["volatilidad"]
daily["banda_inf"] = daily["mediana_precio"] - daily["volatilidad"]

# Ordenar por fecha
daily = daily.sort_values("Fecha").reset_index(drop=True)

# --- Base 100 usando el 29-10-2025 ---
base_fecha = pd.to_datetime("2025-10-29")

base_valor = daily.loc[daily["Fecha"] == base_fecha, "mediana_precio"]

if base_valor.empty:
    raise ValueError("No existe información para la fecha base 29-10-2025.")

base_valor = base_valor.iloc[0]

# Índices base 100
daily["idx"]      = 100 * daily["mediana_precio"] / base_valor
daily["idx_sup"]  = 100 * daily["banda_sup"]      / base_valor
daily["idx_inf"]  = 100 * daily["banda_inf"]      / base_valor

print(daily.head())


In [ ]:
daily

In [ ]:
import plotly.graph_objects as go

# --- Tu DataFrame daily ---
# (pegar aquí tu df si no está en memoria)

fig = go.Figure()

# Línea principal: índice base 100
fig.add_trace(go.Scatter(
    x=daily["Fecha"],
    y=daily["idx"],
    mode="lines",
    name="Índice (Base 100)",
    line=dict(width=2)
))

# # Banda superior
# fig.add_trace(go.Scatter(
#     x=daily["Fecha"],
#     y=daily["idx_sup"],
#     mode="lines",
#     name="Banda superior",
#     line=dict(width=1, dash="dot"),
#     showlegend=True
# ))

# # Banda inferior
# fig.add_trace(go.Scatter(
#     x=daily["Fecha"],
#     y=daily["idx_inf"],
#     mode="lines",
#     name="Banda inferior",
#     line=dict(width=1, dash="dot"),
#     showlegend=True
# ))

# # Relleno entre bandas (sombreado)
# fig.add_trace(go.Scatter(
#     x=pd.concat([daily["Fecha"], daily["Fecha"][::-1]]),
#     y=pd.concat([daily["idx_sup"], daily["idx_inf"][::-1]]),
#     fill="toself",
#     fillcolor="rgba(0, 100, 200, 0.15)",
#     line=dict(color="rgba(0,0,0,0)"),
#     hoverinfo="skip",
#     showlegend=False
# ))

# Layout
fig.update_layout(
    title="Índice de Precio Diario (Base 100 = 29/10/2025) con Bandas de Confianza",
    xaxis_title="Fecha",
    yaxis_title="Índice (Base 100)",
    template="plotly_white",
    hovermode="x unified",
    width=900,
    height=500
)

fig.show()

